In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import torch
from torchvision import datasets, transforms, models
import os
import torch
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import json
from torch import nn, optim


# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode

In [ ]:
train_data = pd.read_csv("../input/cassava-leaf-disease-classification/train.csv")
train_data.head()

In [ ]:
img_id = train_data.iloc[:, 0]
labels = train_data.iloc[:, 1]

In [ ]:
def show_leaf(image, labels):
    """Show image with labels"""
    plt.imshow(image.permute(1, 2, 0))
    plt.title(labels)
    plt.pause(0.001)  # pause a bit so that plots are updated
# plt.figure()
# show_leaf(io.imread(os.path.join('../input/cassava-leaf-disease-classification/train_images', img_id[10])),
#                labels[10])
# plt.show()

In [ ]:
class LeafDataset(Dataset):

    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.train_data = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.train_data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.root_dir,
                                self.train_data.iloc[idx, 0])
        image = io.imread(img_name)
        labels = self.train_data.iloc[idx, 1]
        sample = {'image': image, 'labels': labels}

        if self.transform:
            sample = self.transform(sample)

        return sample

In [ ]:
class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        image, labels = sample['image'], sample['labels']

        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
        image = image.transpose((2, 0, 1))
        return {'image': torch.from_numpy(image),
                 'labels': labels}
    
class Rescale(object):
    """Rescale the image in a sample to a given size.

    Args:
        output_size (tuple or int): Desired output size. If tuple, output is
            matched to output_size. If int, smaller of image edges is matched
            to output_size keeping aspect ratio the same.
    """

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        self.output_size = output_size

    def __call__(self, sample):
        image, labels = sample['image'], sample['labels']

        h, w = image.shape[:2]
        if isinstance(self.output_size, int):
            if h > w:
                new_h, new_w = self.output_size * h / w, self.output_size
            else:
                new_h, new_w = self.output_size, self.output_size * w / h
        else:
            new_h, new_w = self.output_size

        new_h, new_w = int(new_h), int(new_w)

        img = transform.resize(image, (new_h, new_w))

        return {'image': img, 'labels': labels}


class RandomCrop(object):
    """Crop randomly the image in a sample.

    Args:
        output_size (tuple or int): Desired output size. If int, square crop
            is made.
    """

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        if isinstance(output_size, int):
            self.output_size = (output_size, output_size)
        else:
            assert len(output_size) == 2
            self.output_size = output_size

    def __call__(self, sample):
        image, labels = sample['image'], sample['labels']

        h, w = image.shape[:2]
        new_h, new_w = self.output_size

        top = np.random.randint(0, h - new_h)
        left = np.random.randint(0, w - new_w)

        image = image[top: top + new_h,
                      left: left + new_w]

        return {'image': image, 'labels': labels}

In [ ]:
# leaf_dataset = LeafDataset(csv_file='../input/cassava-leaf-disease-classification/train.csv',
#                                     root_dir='../input/cassava-leaf-disease-classification/train_images',
#                                            transform=transforms.Compose([transforms.ToPILImage(),Rescale(245), RandomCrop(224),transforms.ToTensor(),
#                                                                         transforms.Normalize(mean = [0.485,0.456,0.406],std = [0.229,0.224,0.225])]))
leaf_dataset = LeafDataset(csv_file='../input/cassava-leaf-disease-classification/train.csv',
                                    root_dir='../input/cassava-leaf-disease-classification/train_images',
                                           transform=transforms.Compose([Rescale(256), RandomCrop(224), ToTensor()]))

In [ ]:
len(leaf_dataset)

In [ ]:
train_dataset, val_dataset = torch.utils.data.random_split(leaf_dataset, [21000, 397])

dataloader_train = DataLoader(train_dataset, batch_size=50,
                        shuffle=True
                        , num_workers=0)
dataloader_val = DataLoader(val_dataset, batch_size=50,
                        shuffle=True
                        , num_workers=0)

In [ ]:
# fig = plt.figure()

# for i in range(len(leaf_dataset)):
#     sample = leaf_dataset[i]

#     print(i, sample['image'].shape, sample['labels'].shape)
    
#     ax = plt.subplot(1, 4, i + 1)
#     plt.tight_layout()
#     ax.set_title('Sample #{}'.format(i))
#     ax.axis('off')
#     show_leaf(**sample)

#     if i == 3:
#         plt.show()
#         break

In [ ]:
model = models.resnet50(pretrained=True)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
device

In [ ]:
model

In [ ]:
for param in model.parameters():
    param.requires_grad = False
    
from collections import OrderedDict
classifier = nn.Sequential(OrderedDict([('fc1', nn.Linear(2048,1024)),
                                        ('relu', nn.ReLU()),
                                        ('dropout', nn.Dropout(p=0.2)),
                                        ('fc2', nn.Linear(1024,512)),
                                        ('relu2', nn.ReLU()),
                                        ('dropout2', nn.Dropout(p=0.2)),
                                        ('fc3', nn.Linear(512,256)),
                                        ('relu3', nn.ReLU()),
                                        ('dropout4', nn.Dropout(p=0.2)),
                                        ('fc4', nn.Linear(256,5)),
                                       ('output', nn.LogSoftmax(dim=1))]))

model.fc = classifier

In [ ]:
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.fc.parameters(), lr = 0.003)

In [ ]:
epochs = 20
steps = 0
running_loss = 0
print_every = 5

model = model.float()
model.to(device)

for epoch in range(epochs):
    for i in enumerate(dataloader_train):
        steps = steps + 1
        sample = i[1]
        image = sample['image']
        labels = sample['labels']
    #     image = image.view(image.shape[0], -1)
        image = image.float()
        images, labels = image.to(device), labels.to(device)
        optimizer.zero_grad()
        logps = model(images.float())
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()
        running_loss+=loss.item()
        if steps % print_every == 0:
            test_loss = 0
            accuracy = 0
            model.eval()
            with torch.no_grad():
                for j in enumerate(dataloader_val):
                    sample = j[1]
                    image = sample['image']
                    labels = sample['labels']
                    image = image.float()
                    inputs, labels = image.to(device), labels.to(device)
                    logps = model.forward(inputs)
                    batch_loss = criterion(logps, labels)

                    test_loss += batch_loss.item()

                    # Calculate accuracy
                    ps = torch.exp(logps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
                print(f"Epoch {epoch+1}/{epochs}.. "
                f"Train loss: {running_loss/print_every:.3f}.. "
                f"Test loss: {test_loss/len(dataloader_val):.3f}.. "
                f"Test accuracy: {accuracy/len(dataloader_val):.3f}")
                running_loss = 0
                model.train()

In [ ]:
# PATH = 'cassava_net.pth'
# torch.save(model.state_dict(), PATH)

In [ ]:
test_data = pd.read_csv("data/sample_submission.csv")

test_data

test_image = LeafDataset(csv_file='data/sample_submission.csv',
                                    root_dir='data/test_images',
                          transform=transforms.Compose([Rescale(256), RandomCrop(224), ToTensor()]))

test_loader = DataLoader(test_image, batch_size=1,
                        shuffle=True
                        , num_workers=0)

In [ ]:
for i in enumerate(test_loader):
    sample = i[1]
    test_image = sample['image']
    labels = sample['labels']
    x = model(test_image.float())